## __지하철 역 별 유동인구 추정 방법__

----------------------------------------------------------------------
### __Intro__
* 본 ipynb는 집계구 단위로 제공되는 서울시 생활 인구 데이터 전처리 과정을 시각화하는데 목적이 있습니다.
* 실제 유동인구 추정값 데이터를 얻기 위해서는 generate_lp_data.py를 활용하면 됩니다.
* 인구 추정을 위한 생활 인구 데이터의 출처는 아래와 같으며, 본 프로젝트에서 연별로 취합한 데이터는 공식 깃허브 참고 바랍니다.

### __사용 데이터__
* 생활 인구 데이터 집계 범위(집계구): https://data.seoul.go.kr/dataList/OA-14979/S/1/datasetView.do#

### __목적__
* 생활 인구 데이터 기반 지하철 역 유동인구 추정

### __필요성__
* CamelNeon은 서울시에서 제공하는 실시간 도시 데이터(이하, rp)로부터, 지하철 역 내 유동인구 데이터를 수집함.
  * 그러나, rp는 과거 기록을 제공하지 않아 타 데이터와 비교가 불가능함
  * 또한, 유동인구 예측 모델 개발 등 타 개발로 확장이 어려움
* 같은 서울시에서 제공하는 생활 인구 데이터(이하, lp)는 과거 데이터를 제공함.
  * 그러나, 집계구 단위로 제공하기에 본 프로젝트(CamelNeon)에 적용하기 어려움
* 따라서, 본 ipynb는 이전 단계에서 계산된 가중치를 기반으로, lp 기반의 지하철 역 별 유동인구를 추정하는 방법을 소개하고자 함.

### 구조
* 생활인구데이터: 연별 생활인구 데이터가 저장된 폴더(공식 깃허브에서 제공)
* 지하철역 생활인구 추정 및 구축 과정.ipynb: 가중치 기반 유동인구 추정 과정 시각화
* generate_lp_data.py: 가중치 기반 유동인구 추정 python 모듈
* Rate.csv: ./#1 생활인구 데이터 수집/#1 집계구 별 가중치 계산 경로에 있는 Rate.csv와 동일

In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [2]:
# 데이터 용량이 매우 크기에, 효율성을 위해 집계구코드를 제외한 모든 데이터를 32비트로 불러와야함
dtype_spec = {'기준일ID': 'int32', '시간대구분': 'int32',
              '행정동코드': 'int32', '집계구코드': 'int64',
              '총생활인구수': 'float32', '10gen_male': 'float32', 
              '10gen_female': 'float32', '20gen_male': 'float32',
              '20gen_female': 'float32', '30gen_male': 'float32',
              '30gen_female': 'float32', '40gen_male': 'float32', 
              '40gen_female': 'float32', '50gen_male': 'float32', 
              '50gen_female': 'float32', '60gen_male': 'float32', '60gen_female': 'float32'}

In [3]:
target = '군자역' # 서울시 실시간 인구 데이터에서 제공하는 지하철역 44개 중 하나만 입력(본 예시에서는 군자역)
path = './생활인구데이터/' # 공식 깃허브 참고

### 데이터 취합
* __step 1:__ 가중치 데이터에서 target에 해당하는 집계구만 불러오기
* __step 2:__ 생활인구데이터 에서 target에 해당하는 집계구 데이터만 불러와서 시계열로 취합

In [4]:
# Step 1 코드 및 결과
fn_list = os.listdir(path)
region_code = pd.read_csv('Rate.csv')
cols = ['TOT_REG_CD', 'AREA_NM', 'ADM_CD', 'rate']
region_code_edit = region_code.loc[region_code['AREA_NM']==target]
rcl = region_code_edit['TOT_REG_CD'].unique()
rcl = [int(i) for i in rcl] # 집계구 코드만 가져옴
print(rcl)
display(region_code_edit.head()) # 군자역에 해당하는 집계구 및 가중치가 저장된 df

[1105054010001, 1105054010003, 1105054010002, 1105054020024, 1105054030003, 1105054030012, 1105054030008, 1105054030009, 1105055010007, 1105055020004, 1105055020002, 1105055020003, 1105056020032, 1105056020022, 1105056020028, 1105056020002, 1105056020017, 1105056020007, 1105056020003, 1105059010021, 1105059010019, 1105059010013, 1105059010012, 1105059010014, 1105059010016, 1105059010009, 1105059010003]


,AREA_NM,TOT_REG_CD,ADM_NM,rate
154,군자역,1105054010001,군자동,1.000000
155,군자역,1105054010003,군자동,0.009218
156,군자역,1105054010002,군자동,0.489040
157,군자역,1105054020024,군자동,0.000813
158,군자역,1105054030003,군자동,0.760985


In [5]:
# Step 2 코드 및 결과, 2분 정도 소요됨
df = pd.DataFrame()
for fn in tqdm(fn_list):
    LPD_df = pd.read_csv(path + fn, dtype=dtype_spec, index_col=0)
    LPD_df = LPD_df.rename(columns={'기준일ID':'Date'})
    LPD_df = LPD_df[LPD_df['집계구코드'].isin(rcl)]
    LPD_df['Datetime'] = LPD_df['Date'].astype(str) + LPD_df['시간대구분'].astype(str).str.zfill(2) + '0000'
    LPD_df['Datetime'] = pd.to_datetime(LPD_df['Datetime'], format='%Y%m%d%H%M%S')
    df = pd.concat([df, LPD_df])
df.head()

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [01:53<00:00, 22.77s/it]


,Date,시간대구분,집계구코드,총생활인구수,10gen_male,10gen_female,20gen_male,20gen_female,30gen_male,30gen_female,40gen_male,40gen_female,50gen_male,50gen_female,60gen_male,60gen_female,Datetime
295,20200101,16,1105054030009,1121.0,43.0,29.0,130.0,148.0,107.0,97.0,79.0,64.0,70.0,57.0,38.0,67.0,2020-01-01 16:00:00
296,20200101,16,1105055020004,3148.0,109.0,98.0,312.0,354.0,358.0,322.0,227.0,204.0,208.0,227.0,155.0,139.0,2020-01-01 16:00:00
297,20200101,16,1105056020003,595.0,18.0,11.0,53.0,68.0,70.0,56.0,39.0,32.0,52.0,48.0,35.0,35.0,2020-01-01 16:00:00
298,20200101,16,1105056020007,588.0,25.0,24.0,39.0,58.0,57.0,64.0,43.0,40.0,37.0,38.0,26.0,28.0,2020-01-01 16:00:00
299,20200101,16,1105056020028,1874.0,77.0,58.0,139.0,203.0,189.0,194.0,131.0,117.0,126.0,127.0,121.0,105.0,2020-01-01 16:00:00


### 데이터 취합
* step 1: 생활인구df와 가중치df를 집계구코드 컬럼 기준으로 결합 후, Datetime 기준으로 정렬
* step 2: 각 집계구 별 인구수와 집계구 별 가중치를 곱하여 가중치가 적용된 집계구별 인구수 계산
* step 3: Datetime 기준으로 group 한 후 합산

In [6]:
# step 1: 생활인구df와 가중치df를 집계구코드 컬럼 기준으로 결합 후, Datetime 기준으로 정렬
merged_df = pd.merge(df, region_code_edit, how='left', left_on='집계구코드', right_on='TOT_REG_CD')
merged_df.set_index('Datetime', inplace=True)
merged_df.sort_index(ascending=True, inplace=True)
display(merged_df.head())

# step 2: 각 집계구 별 인구수와 집계구 별 가중치를 곱한 새로운 컬럼 생성
gen_columns = [col for col in merged_df.columns if 'gen' in col]
gen_columns.append('총생활인구수')
for col in gen_columns:
    merged_df[f'{col}'] = merged_df[col] * merged_df['rate']
cols = ['총생활인구수']
display(merged_df.head())

# step 3: Datetime 기준으로 group 한 후 합산
for i in range(10,70,10):
    name = f'{i}gen'
    cols.append(f'{name}_male')
    cols.append(f'{name}_female')
final_df = merged_df.groupby(['Datetime'])[cols].sum()
display(final_df.head())

final_df.to_csv(f'{target}_생활인구수.csv', encoding='utf-8')
final_df.head()
print('finish')

,Date,시간대구분,집계구코드,총생활인구수,10gen_male,10gen_female,20gen_male,20gen_female,30gen_male,30gen_female,40gen_male,40gen_female,50gen_male,50gen_female,60gen_male,60gen_female,AREA_NM,TOT_REG_CD,ADM_NM,rate
Datetime,,,,,,,,,,,,,,,,,,,,
2020-01-01,20200101,0,1105059010013,821.0,26.0,18.0,80.0,114.0,99.0,86.0,55.0,43.0,39.0,49.0,39.0,53.0,군자역,1105059010013,능동,0.996482
2020-01-01,20200101,0,1105055020004,1958.0,53.0,72.0,167.0,208.0,254.0,226.0,129.0,131.0,137.0,143.0,71.0,75.0,군자역,1105055020004,중곡1동,0.647339
2020-01-01,20200101,0,1105054030009,584.0,19.0,11.0,77.0,85.0,66.0,60.0,35.0,25.0,37.0,31.0,22.0,30.0,군자역,1105054030009,군자동,1.000000
2020-01-01,20200101,0,1105054030003,78.0,0.0,0.0,5.0,6.0,4.0,0.0,0.0,7.0,0.0,4.0,0.0,5.0,군자역,1105054030003,군자동,0.760985
2020-01-01,20200101,0,1105054010002,538.0,16.0,5.0,79.0,73.0,55.0,55.0,35.0,21.0,39.0,30.0,17.0,27.0,군자역,1105054010002,군자동,0.489040


,Date,시간대구분,집계구코드,총생활인구수,10gen_male,10gen_female,20gen_male,20gen_female,30gen_male,30gen_female,40gen_male,40gen_female,50gen_male,50gen_female,60gen_male,60gen_female,AREA_NM,TOT_REG_CD,ADM_NM,rate
Datetime,,,,,,,,,,,,,,,,,,,,
2020-01-01,20200101,0,1105059010013,818.111478,25.908524,17.936671,79.718536,113.598914,98.651689,85.697426,54.806494,42.848713,38.862786,48.827603,38.862786,52.813530,군자역,1105059010013,능동,0.996482
2020-01-01,20200101,0,1105055020004,1267.489250,34.308953,46.608389,108.105569,134.646458,164.424040,146.298555,83.506697,84.801375,88.685407,92.569440,45.961050,48.550405,군자역,1105055020004,중곡1동,0.647339
2020-01-01,20200101,0,1105054030009,584.000000,19.000000,11.000000,77.000000,85.000000,66.000000,60.000000,35.000000,25.000000,37.000000,31.000000,22.000000,30.000000,군자역,1105054030009,군자동,1.000000
2020-01-01,20200101,0,1105054030003,59.356850,0.000000,0.000000,3.804926,4.565912,3.043941,0.000000,0.000000,5.326897,0.000000,3.043941,0.000000,3.804926,군자역,1105054030003,군자동,0.760985
2020-01-01,20200101,0,1105054010002,263.103707,7.824646,2.445202,38.634187,35.699945,26.897219,26.897219,17.116412,10.269847,19.072574,14.671210,8.313686,13.204089,군자역,1105054010002,군자동,0.489040


,총생활인구수,10gen_male,10gen_female,20gen_male,20gen_female,30gen_male,30gen_female,40gen_male,40gen_female,50gen_male,50gen_female,60gen_male,60gen_female
Datetime,,,,,,,,,,,,,
2020-01-01 00:00:00,7837.761451,223.685544,194.791046,721.829496,927.911515,878.896721,829.997035,545.047497,484.382105,459.055821,562.678437,309.927360,484.593297
2020-01-01 01:00:00,7360.749579,213.396638,190.927691,715.976642,892.604562,848.701147,812.767687,540.304063,440.859284,444.088101,476.322121,297.451458,368.871668
2020-01-01 02:00:00,7189.577050,200.216907,183.625515,718.456905,872.611784,814.571086,783.744745,521.104725,420.168250,427.454548,475.185511,285.290649,351.723358
2020-01-01 03:00:00,7101.092526,216.895865,186.260869,691.763254,874.466327,790.647618,783.125640,513.447334,403.372603,425.421985,453.577290,278.956344,360.111826
2020-01-01 04:00:00,7009.750741,194.391065,197.105007,685.676394,867.638364,789.295734,785.264941,522.256726,399.815951,412.288420,440.754407,266.656000,350.788955


finish
